In [101]:
import pandas as pd
import matplotlib.pyplot as plb
import squarify
import numpy as np
from datetime import timedelta

data = pd.read_csv('../input/superstoredata/us-superstore-data.csv')
data.head()

,Order ID,Order Date,Customer ID,Customer Name,Product ID,Category,Product Name,Sales,Quantity
0,CA-2016-152156,08/11/16,CG-12520,Claire Gute,FUR-BO-10001798,Furniture,Bush Somerset Collection Bookcase,261.9600,2
1,CA-2016-152156,08/11/16,CG-12520,Claire Gute,FUR-CH-10000454,Furniture,Hon Deluxe Fabric Upholstered Stacking Chairs....,731.9400,3
2,CA-2016-138688,12/06/16,DV-13045,Darrin Van Huff,OFF-LA-10000240,Office Supplies,Self-Adhesive Address Labels for Typewriters b...,14.6200,2
3,US-2015-108966,11/10/15,SO-20335,Sean O'Donnell,FUR-TA-10000577,Furniture,Bretford CR4500 Series Slim Rectangular Table,957.5775,5
4,US-2015-108966,11/10/15,SO-20335,Sean O'Donnell,OFF-ST-10000760,Office Supplies,Eldon Fold 'N Roll Cart System,22.3680,2


In [102]:
data.shape

(9994, 9)

In [103]:
data.isnull().count()

Order ID         9994
Order Date       9994
Customer ID      9994
Customer Name    9994
Product ID       9994
Category         9994
Product Name     9994
Sales            9994
Quantity         9994
dtype: int64

In [104]:
data['Order Date'] = pd.to_datetime(data['Order Date'])

data.head()

,Order ID,Order Date,Customer ID,Customer Name,Product ID,Category,Product Name,Sales,Quantity
0,CA-2016-152156,2016-08-11,CG-12520,Claire Gute,FUR-BO-10001798,Furniture,Bush Somerset Collection Bookcase,261.9600,2
1,CA-2016-152156,2016-08-11,CG-12520,Claire Gute,FUR-CH-10000454,Furniture,Hon Deluxe Fabric Upholstered Stacking Chairs....,731.9400,3
2,CA-2016-138688,2016-12-06,DV-13045,Darrin Van Huff,OFF-LA-10000240,Office Supplies,Self-Adhesive Address Labels for Typewriters b...,14.6200,2
3,US-2015-108966,2015-11-10,SO-20335,Sean O'Donnell,FUR-TA-10000577,Furniture,Bretford CR4500 Series Slim Rectangular Table,957.5775,5
4,US-2015-108966,2015-11-10,SO-20335,Sean O'Donnell,OFF-ST-10000760,Office Supplies,Eldon Fold 'N Roll Cart System,22.3680,2


In [105]:
data_customer = data['Customer ID']
data_customer.head()

0    CG-12520
1    CG-12520
2    DV-13045
3    SO-20335
4    SO-20335
Name: Customer ID, dtype: object

In [109]:
snapshot_date = data['Order Date'].max() + timedelta(days=1)

data_process = data.groupby(['Customer ID']).agg({
    'Order Date': lambda x: (snapshot_date - x.max()).days,
    'Order ID': 'count',
    'Sales': 'sum'
})

data_process.rename(columns={
    'Order Date': 'Recency',
    'Order ID': 'Frequency',
    'Sales': 'Monetary'
}, inplace=True)

In [110]:
print(data_process.head())
print('{:,} rows; {:,} columns'
     .format(data_process.shape[0], data_process.shape[1]))

             Recency  Frequency  Monetary
Customer ID                              
AA-10315         185         11  5563.560
AA-10375          49         15  1056.390
AA-10480         260         12  1790.512
AA-10645         234         18  5086.935
AB-10015         446          6   886.156
793 rows; 3 columns


In [111]:
q_recency_1 = np.quantile(data_process['Recency'], 1)
q_recency_2 = np.quantile(data_process['Recency'], 0.75)
q_recency_3 = np.quantile(data_process['Recency'], 0.5)
q_recency_4 = np.quantile(data_process['Recency'], 0.25)

print('Quantile Recency 1 : ', q_recency_1, ' days')
print('Quantile Recency 2 : ', q_recency_2, ' days')
print('Quantile Recency 3 : ', q_recency_3, ' days')
print('Quantile Recency 4 : ', q_recency_4, ' days')

Quantile Recency 1 :  1166  days
Quantile Recency 2 :  219.0  days
Quantile Recency 3 :  97.0  days
Quantile Recency 4 :  38.0  days


In [112]:
q_freq_1 = np.quantile(data_process['Frequency'], 0.25)
q_freq_2 = np.quantile(data_process['Frequency'], 0.5)
q_freq_3 = np.quantile(data_process['Frequency'], 0.75)
q_freq_4 = np.quantile(data_process['Frequency'], 1)

print('Quantile Frequency 1 : ', q_freq_1, ' times')
print('Quantile Frequency 2 : ', q_freq_2, ' times')
print('Quantile Frequency 3 : ', q_freq_3, ' times')
print('Quantile Frequency 4 : ', q_freq_4, ' times')

Quantile Frequency 1 :  8.0  times
Quantile Frequency 2 :  12.0  times
Quantile Frequency 3 :  16.0  times
Quantile Frequency 4 :  37  times


In [113]:
q_monet_1 = np.quantile(data_process['Monetary'], 0.25)
q_monet_2 = np.quantile(data_process['Monetary'], 0.5)
q_monet_3 = np.quantile(data_process['Monetary'], 0.75)
q_monet_4 = np.quantile(data_process['Monetary'], 1)

print('Quantile Monetary 1 : $ ', q_monet_1.round(1))
print('Quantile Monetary 2 : $ ', q_monet_2.round(1))
print('Quantile Monetary 3 : $ ', q_monet_3.round(1))
print('Quantile Monetary 4 : $ ', q_monet_4.round(1))

Quantile Monetary 1 : $  1146.1
Quantile Monetary 2 : $  2256.4
Quantile Monetary 3 : $  3785.3
Quantile Monetary 4 : $  25043.0


In [114]:
r_labels = range(4,0,-1)
r_groups = pd.qcut(data_process['Recency'], q=4, labels=r_labels)

f_labels = range(1,5)
f_groups = pd.qcut(data_process['Frequency'], q=4, labels=f_labels)

m_labels = range(1,5)
m_groups = pd.qcut(data_process['Monetary'], q=4, labels=m_labels)

data_process = data_process.assign(R = r_groups.values, F = f_groups.values, M = m_groups.values)

data_process

,Recency,Frequency,Monetary,R,F,M
Customer ID,,,,,,
AA-10315,185,11,5563.560,2,2,4
AA-10375,49,15,1056.390,3,3,1
AA-10480,260,12,1790.512,1,2,2
AA-10645,234,18,5086.935,1,4,4
AB-10015,446,6,886.156,1,1,1
...,...,...,...,...,...,...
XP-21865,44,28,2374.658,3,4,3
YC-21895,5,8,5454.350,4,1,4
YS-21880,10,12,6720.444,4,2,4


In [116]:
data_process['RFM_score'] = data_process[['R','F','M']].sum(axis=1)

In [117]:
def rfm_level(df):
    if df['RFM_score'] >= 8:
        return 'Loyal'
    elif ((df['RFM_score'] >= 4) and (df['RFM_score'] < 8)):
        return 'Potential'
    else: return 'Needs Attention'
    #elif ((df['RFM_score'] >= 1) and (df['RFM_score'] < 4)):
    #    return 'Needs Attention'
    
data_process['RFM_level'] = data_process.apply(rfm_level, axis=1)
data_process.head()

,Recency,Frequency,Monetary,R,F,M,RFM_score,RFM_level
Customer ID,,,,,,,,
AA-10315,185,11,5563.560,2,2,4,8,Loyal
AA-10375,49,15,1056.390,3,3,1,7,Potential
AA-10480,260,12,1790.512,1,2,2,5,Potential
AA-10645,234,18,5086.935,1,4,4,9,Loyal
AB-10015,446,6,886.156,1,1,1,3,Needs Attention


In [118]:
rfm_level_agg = data_process.groupby('RFM_level').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary': ['mean', 'count']
}).round(1)

print(rfm_level_agg)

                Recency Frequency Monetary      
                   mean      mean     mean count
RFM_level                                       
Loyal              84.2      17.0   4376.1   385
Needs Attention   499.3       5.0    493.3    55
Potential         199.7       9.0   1658.0   353
